In [6]:
import copy
import collections as col
import math
from queue import PriorityQueue
from collections import defaultdict 
################################################ класс вершин ################################################
class Vertex:
    # вершина и смежные с ней вершины и их веса
    def __init__(self,key):
        self.id = key
        self.connectedTo = {}

    def __del__(self):
        del self.id
        del self.connectedTo
    
    # добавление смежной вершины
    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr] = weight
    
    # получение веса с данной вершиной
    def getWeight(self, nbr):
        return self.connectedTo[nbr]
    
    # удаление вершины
    def popVertex(self, key):
        self.connectedTo.pop(key)
     
    # возвращает вершину если она есть в смежных
    def getVertex(self,n):
        return n in self.connectedTo

    
################################################ класс граф ################################################
class Graph:
    def __init__(self):
        self.vertList = {}
        self.oriented = False

    def addVertex(self,key):
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex
            
    def __contains__(self,n):
        return n in self.vertList
    
    def addEdge(self,v,e,w=0,o=False):
        if self.oriented == True:
            if v not in self.vertList:
                nv = self.addVertex(v)
            if e not in self.vertList:
                nv = self.addVertex(e)
            self.vertList[v].addNeighbor(e, w)
        else:
            if v not in self.vertList:
                nv = self.addVertex(v)
            if e not in self.vertList:
                nv = self.addVertex(e)
            self.vertList[v].addNeighbor(e, w)
            self.vertList[e].addNeighbor(v, w)
        
    def delEdge(self,v,e,w=0,o=False):
        if self.oriented == True:
            if self.vertList[v].getVertex(e):
                self.vertList[v].popVertex(e)
                print(f"in {v} delete {e}")
            else:
                print(f"error in {v}")
        else: 
            if self.vertList[v].getVertex(e) and self.vertList[e].getVertex(v):
                self.vertList[v].popVertex(e)
                self.vertList[e].popVertex(v)
                print(f"in {v} delete {e}\nin {e} delete {v}")
            else:
                print(f"error in {v} or {e}")
    
    def delVertex(self, key):
        if self.__contains__(key):
            self.vertList.pop(key)
        print(f"Vertex: {key} is delete")
        for x in self.vertList:
            if self.vertList[x].getVertex(key):
                    self.vertList[x].popVertex(key)

    def getVertices(self):
        return self.vertList.keys()
    
    
    #@classmethod
    def read(self, filename):
        with open(filename) as text:
            n = int(text.read(2))
            ori = int (text.read(2))
            if ori == 1: self.oriented = True
            for i in range(n):
                self.addVertex(i)
            for line in text:
                lst = line.split()
                l = len(lst)
                lst = [int(item) for item in lst]
                if  l == 3:
                    self.addEdge(lst[0], lst[1], lst[2])
                elif l == 2:
                    self.addEdge(lst[0], lst[1])
    #
    def copyVert(self, new):
        for v in self.vertList:
            new[v] = copy.deepcopy(self.vertList[v])
        return new
    
    def write(self, filename):
        with open(filename, 'w') as text:
            n = text.write(f"{len(self.vertList)} ")
            ori = text.write(f'{self.oriented}\n')
            for v in self.vertList:
                for w in self.vertList[v].connectedTo:
                        text.write("%s %s %s" % (v, w, self.vertList[v].getWeight(w)) + '\n')
    
#     # в ширину)
    def bfs (self, s):
        q = col.deque()
        q.append(str(s))
        visited = []
        p=[]
        for i in self.vertList:
            visited.append(i)
            p.append(i) 
            visited[i]=False
        visited[s]=True
        p[s] = -1
        
        while q:
            u = int(q.popleft())
            for to in self.vertList[u].connectedTo:
                if not visited[to]:
                    q.append(str(to))
                    visited[to]=True
                    p[to]=u 
                    
        for i in self.vertList:
            if i == s and not self.vertList[s].getVertex(s):
                visited[i] = False
            if not visited[i]:
                print([])
            else:
                path=[]
                v=i
                while v!=-1:
                    path.append(v)
                    v=p[v]
                path.reverse()
                print(path)
     
    # алгоритм Прима
    def prim (self):
        visited = []
        min_e = []
        best_e = []
        for i in self.vertList:
            visited.append(i) 
            min_e.append(i) 
            best_e.append(i) 
            visited[i] = False
            min_e[i] = math.inf
            
        min_e[0]=0
        
        for i in self.vertList:
            v = -1
            for u in self.vertList:
                if not visited[u] and (v == -1 or min_e[u] < min_e[v]):
                    v = u
            visited[v] = True
            if v != 0:
                print(f"{v} {best_e[v]}")
                
            for e in self.vertList[v].connectedTo:
                u=e
                w=self.vertList[v].connectedTo[e]
                if w<min_e[u]:
                    min_e[u]=w
                    best_e[u]=v
      
    
    def dejkstra(self, s):
        D = [math.inf for i in self.vertList]
        visited = [False for i in self.vertList]
        D[s] = 0

        pq = PriorityQueue()
        pq.put((0, s))

        while not pq.empty():
            (dist, u) = pq.get()
            visited[u]=True

            for e in self.vertList:
                if self.vertList[u].connectedTo.get(e) != None:
                    distance = self.vertList[u].connectedTo[e]
                    if e not in visited:
                        old_cost = D[e]
                        new_cost = D[u] + distance
                        if new_cost < old_cost:
                            pq.put((new_cost, e))
                            D[e] = new_cost
        return D
    
    # есть ли вершина, от которой до остальных минимальный путь не больше n
    def ex_3 (self,n):
        lst = []
        visited = [True for i in self.vertList]
        for v in self.vertList:
            lst = self.dejkstra(v)
            print(lst)
            for i in lst:
                if i > n and i != math.inf:
                    visited[v] = False
                   
        for i in visited:
            if i == True:
                return True
        print("Vertex not found")
    
    def floyd(self):
        g = Graph()
        g=self.copyGraph(g)
        g.oriented=self.oriented
        n = len(g.vertList)
        lst = [k for k in range(n)]
        for v in g.vertList: 
            for k in lst:
                if not g.vertList[v].getVertex(k):
                    g.addEdge(v, k, math.inf)
        del lst
        for k in range(n):
            for i in range(n):
                for j in range(n):
                    g.vertList[i].connectedTo[j] = min (g.vertList[i].connectedTo[j]
                                                       , g.vertList[i].connectedTo[k] + g.vertList[k].connectedTo[j])
        
        return g        
    
    def bellmanFord(self, s): 
        dist = [math.inf for i in self.vertList] 
        dist[s] = 0

        for i in range(len(self.vertList) - 1):
            for u in self.vertList: 
                for v in self.vertList[u].connectedTo:
                    if self.vertList[u].connectedTo.get(v) != None:
                        if dist[u] != math.inf and dist[u] + self.vertList[u].connectedTo[v] < dist[v]: 
                            dist[v] = dist[u] + self.vertList[u].connectedTo[v] 
                        
        for u in self.vertList: 
                for v in self.vertList[u].connectedTo:
                    if self.vertList[u].connectedTo.get(v) != None:
                        if dist[u] != math.inf and dist[u] + self.vertList[u].connectedTo[v] < dist[v]: 
                            print("Graph contains negative weight cycle")
                            return False
#         print(dist)
        return(dist)

      
    def ex_9 (self):
        g = Graph()
        g = self.floyd()
        g.lookAtGraph()  
        
        
    # кратчайшие пути к вершине u   
    def ex_16(self, u):
#         print(self.bellmanFord(u))
        lst = []
        visited =[math.inf for i in self.vertList]
        for v in self.vertList:
            lst=self.bellmanFord(v)
            if lst is False:
                return False
            visited[v]=lst[u]
        print(visited)
        
        
    def bfs_for_ford(self, s, t, parent):
        visited = [False for i in self.vertList]
        queue = []
        queue.append(s)
        visited[s] = True
        while queue:
            u = queue.pop(0)
            for to in self.vertList[u].connectedTo:
                if not visited[to] and self.vertList[u].connectedTo[to] > 0:
                    queue.append(to)
                    visited[to] = True
                    parent[to] = u
        return True if visited[t] else False

    def fordfulkerson(self, s, t):
        g = Graph()
        g = self.copyGraph(g)
        lst = [k for k in g.vertList]
        g.oriented=self.oriented
        for v in g.vertList: 
            for k in lst:
                if not g.vertList[v].getVertex(k):
                    g.addEdge(v, k)
        g.lookAtGraph() 
        
        parent = [-1 for i in self.vertList]
        maxflow = 0
        while g.bfs_for_ford(s, t, parent):
            pathflow = float("Inf")
            t1 = t
            while(t1 != s):
                pathflow = min(pathflow, g.vertList[parent[t1]].connectedTo[t1])
                t1 = parent[t1]
            maxflow += pathflow
            v = t
            while(v != s):
                u = parent[v]
                g.vertList[u].connectedTo[v] -= pathflow
                g.vertList[v].connectedTo[u] += pathflow
                v = parent[v]
        return maxflow
        
################################################ консоль ################################################
    # поиск в глубину 
    def dfs(self, u, v, visited, path):
        visited[u] = True
        path.append(u)
        if u == v:
            print(path)
        else:
            for i in self.vertList[u].connectedTo:
                if visited[i] == False:
                    self.dfs(i, v, visited, path)
        path.pop()
        visited[u] = False

    def vistAll(self, u,v):
        visited = []
        for i in self.vertList:
            visited.append(i) 
            visited[i]=False
        # массив для хранения путей
        path = []
        self.dfs(u, v, visited, path)
              
    def copyGraph(self, newGraph):
        for i in self.vertList:
            newGraph.addVertex(i)
        self.copyVert(newGraph.vertList)
        return newGraph
    
    # просмотр графа
    def lookAtGraph(self):
        for v in self.vertList:
            print("%s:" % v, end=' ')
            if len(self.vertList[v].connectedTo) == 0:
                    print("[]")
            else: 
                c=0
                for w in self.vertList[v].connectedTo:
                    c +=1
                    if  len(self.vertList[v].connectedTo)== c:
                        print("[%s:%s]" % (w, self.vertList[v].getWeight(w)))
                    else: 
                        print("[%s:%s]" % (w, self.vertList[v].getWeight(w)), end =', ') 


    def console(self):
        print("Welcome to Graph!\n\nChoose an action:\n 1. Add vertex\n 2. Delete vertex\n 3. Add edge\n 4. Delete edge\n 5. View\n 6. DFS\n 7. BFS\n")
        try:
            act = int(input('Enter action:'))
            if act == 1:
                v = int(input('Add vertex:'))
                self.addVertex(v)
                print("You added vertex!")
            elif act == 2:
                v = int(input('Delete vertex:'))
                self.delVertex(v)
                print("You delete vertex!")
            elif act == 3:
                v = int(input('Add vertex:'))
                e = int(input('Add adjacent vertex:'))
                w = int(input('Add weight or 0:'))
                self.addEdge(v,e,w)
                print(f"You added edge ({v},{e}) with weight {w}")
            elif act == 4:
                v = int(input('Delete vertex:'))
                e = int(input('Delete adjacent vertex:'))
                self.delEdge(v,e,w)
                print(f"You delete edge ({v},{e})")
            elif act == 5:
                self.lookAtGraph()
            elif act == 6:
                v = int(input('From vertex:'))
                e = int(input('To vertex:'))
                self.vistAll(v,e)
            elif act == 7:
                v = int(input('From vertex:'))
                self.vistAll(v)
            else:
                print("You entered the wrong action")
        except:
            print("You entered not a number")

##### Максимальный поток

In [32]:
g = Graph()
g.read("graph04.txt")
g.lookAtGraph()
print("")
g.fordfulkerson(0,5)

0: [1:16], [2:13]
1: [2:10], [3:12], [0:7]
2: [1:4], [4:14]
3: [2:9], [5:20]
4: [3:7], [5:4]
5: []

0: [1:16], [2:13], [0:0], [3:0], [4:0], [5:0]
1: [2:10], [3:12], [0:7], [1:0], [4:0], [5:0]
2: [1:4], [4:14], [0:0], [2:0], [3:0], [5:0]
3: [2:9], [5:20], [0:0], [1:0], [3:0], [4:0]
4: [3:7], [5:4], [0:0], [1:0], [2:0], [4:0]
5: [0:0], [1:0], [2:0], [3:0], [4:0], [5:0]


23

##### Вывести все кратчайшие пути до вершины u.

In [ ]:
g = Graph()
g.read("graph04.txt")
g.lookAtGraph()
g.ex_16(0)

##### Вывести длины кратчайших путей для всех пар вершин

In [ ]:
g = Graph()
g.read("graph04.txt")
g.lookAtGraph()
print("")
gg = Graph()
gg = g.ex_9()

##### Определить, есть ли в графе вершина, каждая из минимальных стоимостей пути от которой до остальных не превосходит N

In [ ]:
g = Graph()
g.read("graph04.txt")
g.lookAtGraph()
g.ex_3(15)

##### Вывести полустепень исхода данной вершины орграфа.

In [ ]:
g = Graph()
g.read("graph2.txt")
g.lookAtGraph()
v = int(input("Enter vertex: "))
if g.__contains__(v):
    print(f"Vertex {v} has a semi-degree of outcome: {len(g.vertList[v].connectedTo)}")
else:
    print(f"Vertex {v} not found")

##### Вывести все вершины графа, не смежные с данной.

In [ ]:
g = Graph()
g.read("graph2.txt")
g.lookAtGraph()
v = int(input("Enter vertex: "))
lst=[g.vertList[v].id for v in g.vertList]
print(v, end=': ')
for w in g.vertList[v].connectedTo:
    for i in lst:
        if w == i:
            lst.remove(i)
print(lst)

##### Построить орграф, являющийся объединением двух заданных.

In [ ]:
g = Graph()
g.read("graph2.txt")
print("Граф 1:")
g.lookAtGraph()
print("")
gg = Graph()
gg.read("graph3.txt")
print("Граф 2:")
gg.lookAtGraph()
print("")
if len(g.vertList) >= len(gg.vertList):
    for v in g.vertList:
        if gg.__contains__(v):
            g.vertList[v].connectedTo.update(gg.vertList[v].connectedTo)
    g.lookAtGraph()
else: 
    for v in gg.vertList:
        if g.__contains__(v):
            gg.vertList[v].connectedTo.update(g.vertList[v].connectedTo)
    gg.lookAtGraph()

##### Вывести все пути из u в v.

In [ ]:
g = Graph()
g.read("graph2.txt")
g.lookAtGraph()
print("")
g.vistAll(3,5)

##### Вывести кратчайшие пути из данной вершины в остальные

In [7]:
g = Graph()
g.read("graph2.txt")
g.lookAtGraph()
print("")
g.bfs(5)

0: [3:0], [2:0]
1: []
2: [7:0]
3: [8:0], [7:0]
4: [5:0], [7:0], [3:0], [1:0]
5: [7:0]
6: [6:0]
7: [0:0]
8: [1:0]
9: []

[5, 7, 0]
[5, 7, 0, 3, 8, 1]
[5, 7, 0, 2]
[5, 7, 0, 3]
[]
[]
[]
[5, 7]
[5, 7, 0, 3, 8]
[]


##### Алгоритм Прима

In [ ]:
g = Graph()
g.read("graph1.txt")
g.lookAtGraph()
print("")
g.prim()